# Imports needed


In [1]:
!pip install pygame

In [2]:
import math
from itertools import zip_longest
from operator import itemgetter
import random
import pygame

pygame 2.1.2 (SDL 2.0.18, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


### To run pygame


# Temperature Function
Using s-curve

In [3]:
# temperature function - s-curve
def temperature(i):
    amplitude = 1000
    center = 0
    width = 0.11 * iterations
    return float(amplitude * (1 / (1 + math.exp((i - center) / width))))

# Dataset


## Construct job shop problem


In [4]:
# example dataset

schedule_example = [[[(0, 0, 0, 20), (0, 1, 3, 87), (0, 2, 1, 31), (0, 3, 4, 76), (0, 4, 2, 17)], 
                     [(1, 0, 4, 25), (1, 1, 2, 32), (1, 2, 0, 24), (1, 3, 1, 18), (1, 4, 3, 81)], 
                     [(2, 0, 1, 72), (2, 1, 2, 23), (2, 2, 4, 28), (2, 3, 0, 58), (2, 4, 3, 99)], 
                     [(3, 0, 2, 86), (3, 1, 1, 76), (3, 2, 4, 97), (3, 3, 0, 45), (3, 4, 3, 90)], 
                     [(4, 0, 4, 27), (4, 1, 0, 42), (4, 2, 3, 48), (4, 3, 2, 17), (4, 4, 1, 46)], 
                     [(5, 0, 1, 67), (5, 1, 0, 98), (5, 2, 4, 48), (5, 3, 3, 27), (5, 4, 2, 62)], 
                     [(6, 0, 4, 28), (6, 1, 1, 12), (6, 2, 3, 19), (6, 3, 0, 80), (6, 4, 2, 50)], 
                     [(7, 0, 1, 63), (7, 1, 0, 94), (7, 2, 2, 98), (7, 3, 3, 50), (7, 4, 4, 80)], 
                     [(8, 0, 4, 14), (8, 1, 0, 75), (8, 2, 2, 50), (8, 3, 1, 41), (8, 4, 3, 55)], 
                     [(9, 0, 4, 72), (9, 1, 2, 18), (9, 2, 1, 37), (9, 3, 3, 79), (9, 4, 0, 61)]]];

# Algorithm Functions


In [5]:
def find_prev_index(jobs, job):
    job_n = job[0]
    job_seq = job[1] - 1
    if job_seq < 0:
        return None
    for i in range(0, len(jobs)):
        if jobs[i][0] == job_n and jobs[i][1] == job_seq:
            return i


# generating a random, valid neighbor based on previous solution
def shuffle_jobs(array):
    while True:
        first_job, second_job = random.choice(array), random.choice(array)
        # check if job_number is different
        if first_job[0] != second_job[0]:
            # indexes of two elements
            a, b = array.index(first_job), array.index(second_job)
            # indexes of previous jobs
            a_prev, b_prev = find_prev_index(array, first_job), find_prev_index(array, second_job)
            # check if the order of jobs are still valid
            if (a_prev is None or a_prev <= b) and (b_prev is None or b_prev <= a):
                array[b], array[a] = array[a], array[b]  # swap jobs
                return array


# evaluation function, total length of schedule
def find_max_duration(array):
    result = 0
    array = [x for x in array if x != []]
    for sublist in array:
        temp = max(sublist, key=itemgetter(2))[2]
        if result < temp:
            result = temp
    return result


# creates schedule out of a ordered job list
# each job list is validated before
def schedule(jobs, machine_count):
    # first parameter is machine number
    results = [[] for x in range(machine_count)]
    time = [0] * (len(jobs) + 1)  # time of end of job
    time_m = [0] * machine_count  # time end of machine last job
    for job in jobs:
        job_n = job[0]  # job number
        machine_n = job[2]  # machine number
        duration = job[3]  # duration length
        # starting time is max(end of prev task in job, end of prev task on machine)
        start = max(time_m[machine_n], time[job_n])
        results[machine_n].append((job, start, start + duration))  # insert to the end
        # update the last time of job and machine
        time[job_n] = start + duration  # set new time of end of job
        time_m[machine_n] = start + duration    # set new end of last job of machine
    return results


# function that prints schedule as readable output
def print_result(res):
    i = 0
    for machine in res:
        string = "machine " + str(i) + ":"
        for job in machine:
            string = string + "(" + str(job[0][0]) + ", " + str(job[0][1]) + ", " + str(job[1]) + ")"
        print(string)
        i += 1

# Data Visualization

In [6]:
# used to visualize a single schedule

def show(schedule, am_jobs):

    # window initialization
    pygame.init()
    dis_x = 1800
    game_display = pygame.display.set_mode((dis_x, 900))    # change if problem with window size
    game_display.fill((0, 0, 0))

    scaling = (dis_x*0.99)/find_max_duration(schedule)  # scaling factor, so each schedule will fit
    height = int(600 / len(schedule))   # total height the same every time, single height depends on amount of machines
    start_x = 10
    y = 150

    # random color list, to assign each job a different color
    colors = [(random.randint(0, 240), random.randint(0, 240), random.randint(0, 240)) for i in range(50)]

    # calculate size and position of each task and create corresponding rectangle
    for machine in schedule:
        x = start_x + (machine[0][1] * scaling)
        for task in machine:
            if machine.index(task) + 1 < len(machine):
                next = machine[machine.index(task) + 1]
                width = next[1] - task[1]
            pygame.draw.rect(game_display, colors[task[0][0]], (x, y, int(task[0][3] * scaling), height))
            x += width * scaling
        y += height

    # window loop
    while True:
        for event in pygame.event.get():

            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
        
        width = 300
        height = 300

# create the display window
        #win = pygame.display.set_mode((width, height))
        #white = (255, 255, 255)
        #win.fill(white)
        #pygame.image.save(win, "image.jpg")
        pygame.display.update()

# Run 


In [7]:
%%time
def run():
    #tests = construct_tests_js1()       # get benchmarks in jobshop1.txt
    #tests = schedule
    jobs_data = schedule_example[benchmark_id]         # choose the benchmark you want to run

    jobs_data = [list(filter(None, i)) for i in zip_longest(*jobs_data)]

    
    average_makespan = 0
    count = 0

    first_solution = 0
    best_duration = int
    best_jobs = []
    
    do_algo_best_jobs = []
    do_algo_best_duration = 1000000000000000000000000000000

    for x in range(do_algo):

        # order and flatten list
        jobs = []
        for sublist in jobs_data:
            for item in sublist:
                jobs.append(item)

        #machines_count = max(jobs, key=itemgetter(2))[2] + 1  # find how many machines are there
        machines_count  = 5
        results = schedule(jobs, machines_count)      # generate first solution
        min_duration = find_max_duration(results)     # calculate duration of first solution
        first_solution = min_duration

        best_duration = min_duration + 1
        current_best = results       # set first solution as best_solution

        for i in range(iterations):
            old = list(results)     # previous schedule
            alter = shuffle_jobs(jobs)      # new job list
            results = schedule(alter, machines_count)  # calculate schedule of new solution

            # calculate duration of old and new schedule
            d_old = find_max_duration(old)
            d_alter = find_max_duration(results)

            if d_old >= d_alter:
                jobs = alter
            elif random.uniform(0, 1) < math.exp(-(d_alter - d_old) / temperature(i)):
                jobs = alter

            if d_alter < min_duration:
                min_duration = d_alter
                current_best = list(results)
                
        count +=1;    
        #print("Min Duration for iteration"+str(count)+": "+str(min_duration));
        #print("Previoues Best Duration for iteration"+str(count)+": "+str(best_duration));
        
        average_makespan = average_makespan + min_duration
        if best_duration > min_duration:
            best_duration = min_duration
            best_jobs = current_best
        
        if do_algo_best_duration > min_duration:
            do_algo_best_duration = min_duration
            do_algo_best_jobs = current_best
        
        #print("After Best Duration for iteration"+str(count)+": "+str(best_duration));

    print("first solution: " + str(first_solution))
    print("minimum duration: " + str(do_algo_best_duration))
    print("average duration:" + str(average_makespan/do_algo))

    print_result(do_algo_best_jobs)
    if visualisation:
        show(best_jobs, len(jobs_data))



Wall time: 0 ns


In [ ]:
%%time

visualisation = True  # maybe turn of if do_algo > 1
iterations = 1000 # amount of iterations, while one iteration of the algorithm
do_algo = 50  # iterations of applying the algorithm
benchmark_id = 0  # choose number between 0 and 81, check tests.txt

import time
start_time = time.time()
run()
print("--- %s seconds ---" % (time.time() - start_time))


first solution: 904
minimum duration: 702
average duration:768.74
machine 0:(0, 0, 0)(7, 1, 20)(2, 3, 114)(6, 3, 183)(8, 1, 263)(3, 3, 338)(5, 1, 393)(4, 1, 491)(1, 2, 533)(9, 4, 559)
machine 1:(2, 0, 0)(6, 1, 72)(7, 0, 164)(9, 2, 227)(5, 0, 264)(4, 4, 331)(0, 2, 377)(3, 1, 408)(8, 3, 643)(1, 3, 684)
machine 2:(2, 1, 72)(3, 0, 95)(1, 1, 181)(5, 4, 331)(6, 4, 393)(7, 2, 443)(9, 1, 541)(4, 3, 559)(0, 4, 576)(8, 2, 593)
machine 3:(0, 1, 20)(7, 3, 114)(6, 2, 164)(4, 2, 183)(5, 3, 231)(9, 3, 264)(2, 4, 343)(8, 4, 442)(3, 4, 497)(1, 4, 587)
machine 4:(4, 0, 0)(6, 0, 27)(9, 0, 55)(8, 0, 127)(1, 0, 141)(2, 2, 172)(3, 2, 200)(0, 3, 408)(7, 4, 541)(5, 2, 621)
